<a href="https://colab.research.google.com/github/katie291100/learn-nlp/blob/main/0_33acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00


In [2]:
import os

folder_path = 'drive/MyDrive/storypoint/IEEE TSE2018/dataset'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
print(csv_files)

['talendesb.csv', 'springxd.csv', 'clover.csv', 'aptanastudio.csv', 'datamanagement.csv', 'mulestudio.csv', 'moodle.csv', 'jirasoftware.csv', 'talenddataquality.csv', 'bamboo.csv', 'duracloud.csv', 'mesos.csv', 'usergrid.csv', 'titanium.csv', 'mule.csv', 'appceleratorstudio.csv']


In [ ]:
import torch

gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")
device=torch.device('cuda')

In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer, BertModel
from torch import nn
import torch.nn.functional as nnf

# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 5
batch_size = 32
num_epochs = 5
learning_rate = 3e-5
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)




    def forward(self, input_ids, attention_mask):
            output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            x = self.fc1(output.pooler_output)
            x=self.dropout(x)
            x = nnf.relu(self.fc2(x))

            x=self.dropout(x)
            logits = nnf.relu(self.fc3(x))
            return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

In [ ]:
import pandas as pd

dataframes = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
combined_df

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np
from sklearn.model_selection import train_test_split

combined_df = combined_df.query("description.notna()").get(["storypoint", "description", "title"])
df1 = combined_df.query("storypoint in [1]")[:2000]
df2 = combined_df.query("storypoint in [2]")[:2000]
df3 = combined_df.query("storypoint in [3]")[:2000]
df4 = combined_df.query("storypoint in [5]")[:2000]
df5 = combined_df.query("storypoint in [8]")[:2000]

combined_df = df1.append(df2).append(df3).append(df4).append(df5)
combined_df['description'] = combined_df['title'] + ' ' + combined_df['description']
combined_df = combined_df.drop(columns=['title'])
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

# combined_df["storypoint"].value_counts()

In [ ]:
combined_df

In [ ]:
torch.cuda.empty_cache()

In [ ]:
combined_df["storypoint"].value_counts()

In [ ]:
from torch.utils.data import DataLoader, Dataset

class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
          super().__init__()
          self.texts = texts
          self.labels = np.array(list(map(lambda x: [1,2,3,5,8].index(x), list(labels))))
          self.tokenizer = tokenizer
          self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = torch.tensor(self.labels[idx]).to(device)
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      encoding = encoding.to("cuda")
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': label}

train, testds = train_test_split(combined_df, test_size=0.2)
print(len(train['storypoint'].values))
print(len(testds['description'].values))

train_dataset = TextClassificationDataset(train['description'].values, train['storypoint'].values, tokenizer, 256)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [ ]:

print(train['storypoint'].values)
print(np.array(list(map(lambda x: [1,2,3,5,8].index(x), list(train['storypoint'].values)))))

In [ ]:
from sklearn.metrics import classification_report
train_dataset = TextClassificationDataset(train['description'].values, train['storypoint'].values, tokenizer, 256)
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

test_dataset = TextClassificationDataset(testds['description'].values, testds['storypoint'].values, tokenizer, 256)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
def test():
  model.eval()
  predictions = []
  true_labels = []

  model.eval()
  predictions = []
  actual_labels = []
  with torch.no_grad():
      for i, batch in enumerate(test_dataloader):
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)
          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          _, preds = torch.max(outputs, dim=1)
          predictions.extend(preds.cpu().tolist())
          actual_labels.extend(labels.cpu().tolist())

  print(classification_report(actual_labels, predictions))

# loss = 0
for epoch in range(num_epochs):
  model.train()
  epoch_loss = 0

  for i, batch in enumerate(data_loader):
      optimizer.zero_grad()
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = nn.CrossEntropyLoss()(outputs, labels)
      loss.backward()
      epoch_loss += loss.item()
      optimizer.step()
  print("loss: ",epoch_loss/len(data_loader))
  test()



In [ ]:
def test():
  model.eval()
  predictions = []
  true_labels = []

  model.eval()
  predictions = []
  actual_labels = []
  with torch.no_grad():
      for i, batch in enumerate(test_dataloader):
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)
          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          preds = torch.argmax(outputs, dim=1)
          predictions.extend(preds.cpu().tolist())
          actual_labels.extend(labels.cpu().tolist())


  print(classification_report(actual_labels, predictions))

test()

In [ ]:
3e-4

In [ ]:
report = classification_report(actual_labels, predictions)
print(report)
print(true_labels)
print(predictions)